# <font color=blue>Assignments for "Overfitting and Regularization"</font>

In this assignment, you'll continue working with the house prices data. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import mse, rmse

import warnings
warnings.filterwarnings(action="ignore")

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/house_prices.csv", sep = ";")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


- Reimplement your model from the previous lesson.

In [3]:
df_null_ratios = df.isna().sum() / len(df)
df.drop(df_null_ratios[df_null_ratios > .1].index, axis=1, inplace=True)
df.dropna(inplace=True)

#X_num = df[['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF']]
X_num = df.select_dtypes(exclude='O').drop(['SalePrice', 'Id'], axis=1)

ExterQual = pd.get_dummies(df['ExterQual'], drop_first=True)
BsmtQual = pd.get_dummies(df['BsmtQual'], drop_first=True)
BsmtFinType1 = pd.get_dummies(df['BsmtFinType1'], drop_first=True)
KitchenQual = pd.get_dummies(df['KitchenQual'], drop_first=True)
GarageQual = pd.get_dummies(df['GarageQual'], drop_first=True)
GarageCond = pd.get_dummies(df['GarageCond'], drop_first=True)

X_cat = pd.concat([ExterQual, BsmtQual, BsmtFinType1, KitchenQual, GarageQual, GarageCond], axis=1)

scaler = StandardScaler()

X_num = pd.DataFrame(scaler.fit_transform(X_num), columns=X_num.columns, index=X_num.index)

X = pd.concat([X_num, X_cat], axis=1)
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [4]:
lrm = LinearRegression()

def print_results(model):
  model.fit(X_train, y_train)
  y_preds_train = model.predict(X_train)
  y_preds_test = model.predict(X_test)

  print("R-squared of the model in training set is: {}".format(model.score(X_train, y_train)))
  print("-----Test set statistics-----")
  print("R-squared of the model in test set is: {}".format(model.score(X_test, y_test)))
  print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
  print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
  print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
  print("Mean absolute percentage error of the prediction is: {}\n".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

print_results(model=LinearRegression())

R-squared of the model in training set is: 0.9151039983821223
-----Test set statistics-----
R-squared of the model in test set is: -4.5167418599999524e+18
Mean absolute error of the prediction is: 9847255091736.584
Mean squared error of the prediction is: 1.9490654911228322e+28
Root mean squared error of the prediction is: 139608935642487.88
Mean absolute percentage error of the prediction is: 7947666621.9213915



- Try OLS, Lasso, Ridge and ElasticNet regressions using the same model specification. This time, you need to do **k-fold cross-validation** to choose the best hyperparameter values for your models. Which model is the best? Why?

In [5]:
lasso = Lasso()
print('Lasso Regression:')
print_results(model=lasso)

ridge = Ridge()
print('Ridge Regression:')
print_results(model=ridge)

elastic = ElasticNet()
print('ElasticNet Regression:')
print_results(model=elastic)

Lasso Regression:
R-squared of the model in training set is: 0.9151011980510864
-----Test set statistics-----
R-squared of the model in test set is: 0.2804689948031821
Mean absolute error of the prediction is: 23076.38115897683
Mean squared error of the prediction is: 3104921856.2648954
Root mean squared error of the prediction is: 55721.82567239605
Mean absolute percentage error of the prediction is: 13.740705689183457

Ridge Regression:
R-squared of the model in training set is: 0.9143518577406746
-----Test set statistics-----
R-squared of the model in test set is: 0.2826218408331147
Mean absolute error of the prediction is: 22828.879244803753
Mean squared error of the prediction is: 3095631890.1018915
Root mean squared error of the prediction is: 55638.403015380405
Mean absolute percentage error of the prediction is: 13.522651855302003

ElasticNet Regression:
R-squared of the model in training set is: 0.8681359006444056
-----Test set statistics-----
R-squared of the model in test se

In [6]:
params = {'alpha': [10**i for i in range(1, 10)] + [round(0.1**i, 10) for i in range(10)]}

lasso_grid = GridSearchCV(Lasso(), param_grid=params, cv=10)

lasso_grid.fit(X_train, y_train)
print(lasso_grid.best_params_)

{'alpha': 10}


In [7]:
pd.DataFrame(lasso_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

,param_alpha,mean_test_score
0,10,0.887788
1,100,0.887748
9,1,0.885996
10,0.1,0.885733
11,0.01,0.885706


In [8]:
ridge_grid = GridSearchCV(Ridge(), param_grid=params, cv=10)

ridge_grid.fit(X_train, y_train)
print(ridge_grid.best_params_)

{'alpha': 1.0}


In [9]:
pd.DataFrame(ridge_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

,param_alpha,mean_test_score
9,1,0.888322
10,0.1,0.888020
11,0.01,0.887635
12,0.001,0.887556
13,0.0001,0.887547


In [10]:
elastic_grid = GridSearchCV(ElasticNet(), param_grid=params, cv=10)

elastic_grid.fit(X_train, y_train)
print(elastic_grid.best_params_)

{'alpha': 0.01}


In [11]:
pd.DataFrame(elastic_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

,param_alpha,mean_test_score
11,0.01,0.888244
12,0.001,0.888237
13,0.0001,0.887834
14,1e-05,0.887523
15,1e-06,0.886233


In [12]:
lasso = Lasso(alpha=1000)
print('Lasso Regression:')
print_results(model=lasso)

ridge = Ridge(alpha=1000)
print('Ridge Regression:')
print_results(model=ridge)

elastic = ElasticNet(alpha=1)
print('ElasticNet Regression:')
print_results(model=elastic)

Lasso Regression:
R-squared of the model in training set is: 0.8944043035930861
-----Test set statistics-----
R-squared of the model in test set is: 0.3310763104525577
Mean absolute error of the prediction is: 22631.985588835734
Mean squared error of the prediction is: 2886541050.835029
Root mean squared error of the prediction is: 53726.53953899347
Mean absolute percentage error of the prediction is: 13.667050857762518

Ridge Regression:
R-squared of the model in training set is: 0.8396898820204578
-----Test set statistics-----
R-squared of the model in test set is: 0.5640801145312861
Mean absolute error of the prediction is: 22317.781544516856
Mean squared error of the prediction is: 1881082497.0663621
Root mean squared error of the prediction is: 43371.4479475422
Mean absolute percentage error of the prediction is: 13.206331791129763

ElasticNet Regression:
R-squared of the model in training set is: 0.8681359006444056
-----Test set statistics-----
R-squared of the model in test set 